## Setting up the environment

In [1]:
!pip install hydra-core==1.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 144 kB 34.5 MB/s 
     |████████████████████████████████| 74 kB 3.9 MB/s 
     |████████████████████████████████| 112 kB 79.7 MB/s 
     |████████████████████████████████| 596 kB 48.7 MB/s 
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.whl size=141230 sha256=74559b8666993b801a0d5c420d290a1f61d2125b70bd137e2e59ad9fbeeb929e
  Stored in directory: /root/.cache/pip/wheels/ca/33/b7/336836125fc9bb4ceaa4376d8abca10ca8bc84ddc824baea6c
Successfully built antlr4-python3-runtime
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
!pip install scipy==1.7.3

BRANCH = 'main'
# If you're using Google Colab and not running locally, uncomment and run this cell.
!apt-get install sox libsndfile1 ffmpeg
!pip install wget unidecode pynini==2.1.4
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]

#=== load the repo and data (Thanks Synthbot) ===
!sudo dpkg --configure -a
!sudo apt -qq install -y sox
!git clone "https://github.com/synthbot-anon/synthbot.git" /content/synthbot
!(cd /content/synthbot; git checkout experimental)
!python3 -m pip install pysoundfile 
!python3 -m pip install librosa

!jupyter nbextension enable --py --sys-prefix widgetsnbextension

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libsndfile1 is already the newest version (1.0.28-4ubuntu0.18.04.2).
ffmpeg is already the newest version (7:3.4.11-0ubuntu0.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libmagic-mgc libmagic1 libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa
  libsox-fmt-base libsox3
Suggested packages:
  file libsox-fmt-all
The following NEW packages will be installed:
  libmagic-mgc libmagic1 libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa
  libsox-fmt-base libsox3 sox
0 upgraded, 8 newly installed, 0 to remove and 49 not upgraded.
Need to get 760 kB of archives.
After this operation, 6,717 kB of additional disk space will be used.
Get:1 http://archive

In [3]:
home_path = !(echo $HOME)
home_path = home_path[0]
print(home_path)

/root


In [4]:
#downloading the dataset
id='16P8NctJO_xt8NzfCaEEKV8i_3GSAfu8F'
character='trixie'
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id={id}' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id={id}" -O {character+'.tar'} && rm -rf /tmp/cookies.txt


--2022-07-27 09:10:36--  https://docs.google.com/uc?export=download&confirm=t&id=16P8NctJO_xt8NzfCaEEKV8i_3GSAfu8F
Resolving docs.google.com (docs.google.com)... 74.125.24.138, 74.125.24.102, 74.125.24.100, ...
Connecting to docs.google.com (docs.google.com)|74.125.24.138|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-10-84-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/sernejamo8g7n7jrlo8annok399fantj/1658913000000/05434657448262868821/*/16P8NctJO_xt8NzfCaEEKV8i_3GSAfu8F?e=download&uuid=fa7927f1-3df1-4dd0-861b-9b93e89eceee [following]
--2022-07-27 09:10:38--  https://doc-10-84-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/sernejamo8g7n7jrlo8annok399fantj/1658913000000/05434657448262868821/*/16P8NctJO_xt8NzfCaEEKV8i_3GSAfu8F?e=download&uuid=fa7927f1-3df1-4dd0-861b-9b93e89eceee
Resolving doc-10-84-docs.googleusercontent.com (doc-10-84-docs.googleusercontent.com)... 74.125.24.132, 2404:

In [5]:
skip_noisy = True # Disable to train with Noisy Data included
allowed_emotions = """
Anxious
Angry
Annoyed
Amused
Confused
Crazy
Disgust
Exhausted
Fear
Happy
Neutral
Sad
Serious
Singing
Shouting
Surprised
Smug
Love
Sarcastic
Tired
Whispering
Whining
""".split("\n")[1:-1]

percentage_training_data = 0.95 # 90% of Data will be used for training, 5% for Validation

##############################################################################################

import sys
sys.path.append('/content/synthbot/src')
from ponysynth.corpus import ClipperArchive, phoneme_transcription
import librosa
import subprocess

archive_fn = "./trixie.tar"
archive = ClipperArchive(archive_fn)

data_path = 'trixie'
allowed_emotions = [x.lower() for x in allowed_emotions]
!mkdir {data_path}
!mkdir {data_path+"/out"}
all_clips = []; all_clips_arpa = []; skipped_count=0; too_short_count=0; emotion_skip=0
for key in archive.keys(): # write the audio files for processing in bash terminal
  audio = archive.read_audio(key)
  audio_fn = '{}/{}.wav'.format(data_path, key)
  audio_fn_ = '{}/{}.wav'.format(data_path+"/out", key)
  with open(audio_fn, 'wb') as audio_out:
    audio_out.write(audio.read())

/usr/local/lib/python3.7/dist-packages/resampy/interpn.py:114: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  _resample_loop_p(x, t_out, interp_win, interp_delta, num_table, scale, y)


In [6]:
%%script bash
# trim all 48Khz files
cd trixie
for input in *.wav; do
  output="out/$input"
  sox "$input" "$output" silence 1 0.05 0.1% reverse silence 1 0.05 0.1% reverse;
done

In [7]:
import os

should_continue = 0 # should run "continue" command inside outer loop.
for key in archive.keys():
  label = archive.read_label(key)
  audio_fn = '{}/{}.wav'.format(data_path, key)
  audio_fn_ = '{}/{}.wav'.format(data_path+"/out", key)
  if (label['noise'] in ['Very Noisy','Noisy']) and skip_noisy: os.remove(audio_fn_); skipped_count+=1; continue

  for tag in label['tags']:
     if tag.lower() not in allowed_emotions:
       try: os.remove(audio_fn_)
       except: pass
       print(tag+" emotion not in list"); emotion_skip+=1; should_continue = 1; break # this is supposed to break the outer loop
  if should_continue: should_continue = 0; continue

  audio = archive.read_audio(key)
  transcript = label['utterance']['content']
  
  if os.stat(audio_fn_).st_size < 71602:
    #print("Skipping Audio, Duration: "+str(len(librosa.core.load(audio_fn_, sr=48000)[0])/48000))
    try: os.remove(audio_fn_)
    except: pass
    too_short_count+=1; continue # Skips files based on size.

  filelist_line = "{}|{}".format(audio_fn_, transcript)
  all_clips.append(filelist_line)
  filelist_line = "{}|{}".format(audio_fn_, phoneme_transcription(label))
  all_clips_arpa.append(filelist_line)
    
print(str(skipped_count)+" Files are too Noisy.")
print(str(emotion_skip)+" Files contain an emotion not in permitted emotions")
print(str(too_short_count)+" Files are too short")
print(str(len(list(archive.keys()))-(skipped_count+too_short_count+emotion_skip))+" Files kept in dataset.")


####################################################################################################################
# shuffle the training data
import random
random.seed(0)
random.shuffle(all_clips)

# get train, test, validation splits
num_clips = len(all_clips)
train_end = int(num_clips * percentage_training_data)

train = all_clips[:train_end]
validation = all_clips[train_end:]

train_arpa = all_clips_arpa[:train_end]
validation_arpa = all_clips_arpa[train_end:]

# dump the info to filelist files
with open('train_filelist.txt', 'w') as train_out:
  train_out.write('\n'.join(train)+'\n'+'\n'.join(train_arpa)+"")
with open('val_filelist.txt', 'w') as val_out:
  val_out.write('\n'.join(validation)+'\n'+'\n'.join(validation_arpa)+"")

##################################################################################################################

#imports
import os
import json
import random
import librosa

import torch
import IPython.display as ipd
from matplotlib.pyplot import imshow
from matplotlib import pyplot as plt
from pathlib import Path

from nemo.collections.tts.models import FastPitchModel

######################################################################################

#creating the json files for training and validation
with open('/content/train_filelist.txt') as f :
  records = f.readlines()

print("Number of records : ",len(records))

train_manifest = 'fastpitch_train.json'

train_rec = []
random.shuffle(records)
count = 0

for i in records :
  #if count > 1200 :
  #  break
  
  i = i.split('|')
  audio_filepath = i[0]
  text = i[-1].strip('\n')
  if '{' in text :
    #print(text)
    continue
  count = count + 1

  duration = librosa.get_duration(filename=audio_filepath)
  r = {
       "audio_filepath" : audio_filepath,
       "text" : text,
       "duration" : round(duration,1),
       "text_no_preprocessing" : text
    }

  train_rec.append(r)

with open(train_manifest, "w") as f:
    for s in train_rec:
        f.write(json.dumps(s) + '\n')
        
print("Training Data : ", len(train_rec))

##########################################################################################

with open('/content/val_filelist.txt') as f :
  records_val = f.readlines()

print("Number of records : ", len(records_val))

val_manifest = 'fastpitch_val.json'
count = 0
val_rec = []
random.shuffle(records_val)

for i in records_val:
  #if count > 50 :
  #  break
    
  i = i.split('|')
  audio_filepath = i[0]
  text = i[-1].strip('\n')

  if '{' in text :
    #print(text)
    continue
  count = count + 1
  duration = librosa.get_duration(filename=audio_filepath)
  r = {
       "audio_filepath" : audio_filepath,
       "text" : text,
       "duration" : round(duration,1),
       "text_no_preprocessing" : text
    }

  val_rec.append(r)

print(len(val_rec))
with open(val_manifest, "w") as f:
    for s in val_rec:
        f.write(json.dumps(s) + '\n')

####################################################################################

FastPitchModel.from_pretrained("tts_en_fastpitch")
nemo_files = [p for p in Path(f"{home_path}/.cache/torch/NeMo/").glob("**/tts_en_fastpitch_align.nemo")]
print(f"Copying {nemo_files[0]} to ./")
Path("./tts_en_fastpitch_align.nemo").write_bytes(nemo_files[0].read_bytes())

###################################################################################

!wget https://raw.githubusercontent.com/nvidia/NeMo/main/examples/tts/fastpitch_finetune.py

!mkdir -p conf \
&& cd conf \
&& wget https://raw.githubusercontent.com/nvidia/NeMo/main/examples/tts/conf/fastpitch_align_v1.05.yaml \
&& cd ..

#######################################
# additional files
!mkdir -p tts_dataset_files && cd tts_dataset_files \
&& wget https://raw.githubusercontent.com/NVIDIA/NeMo/main/scripts/tts_dataset_files/cmudict-0.7b_nv22.07 \
&& wget https://raw.githubusercontent.com/NVIDIA/NeMo/main/scripts/tts_dataset_files/heteronyms-030921 \
&& wget wget https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/nemo_text_processing/text_normalization/en/data/whitelist/lj_speech.tsv \
&& cd ..



509 Files are too Noisy.
0 Files contain an emotion not in permitted emotions
18 Files are too short
499 Files kept in dataset.


[NeMo W 2022-07-27 09:11:06 optimizers:55] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2022-07-27 09:11:07 experimental:28] Module <class 'nemo.collections.tts.torch.tts_tokenizers.IPATokenizer'> is experimental, not ready for production and is not fully supported. Use at your own risk.


Number of records :  948
Training Data :  474
Number of records :  50
25
[NeMo I 2022-07-27 09:11:07 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/tts_en_fastpitch/versions/1.8.1/files/tts_en_fastpitch_align.nemo to /root/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_en_fastpitch_align/26d7e09971f1d611e24df90c7a9d9b38/tts_en_fastpitch_align.nemo
[NeMo I 2022-07-27 09:11:24 common:789] Instantiating model from pre-trained checkpoint
[NeMo I 2022-07-27 09:11:27 tokenize_and_classify:87] Creating ClassifyFst grammars.


[NeMo W 2022-07-27 09:11:48 experimental:28] Module <class 'nemo.collections.tts.torch.g2ps.IPAG2P'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2022-07-27 09:11:49 g2ps:87] apply_to_oov_word=None, This means that some of words will remain unchanged if they are not handled by any of the rules in self.parse_one_word(). This may be intended if phonemes and chars are both valid inputs, otherwise, you may see unexpected deletions in your input.
[NeMo W 2022-07-27 09:11:49 modelPT:143] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.torch.data.TTSDataset
      manifest_filepath: /ws/LJSpeech/nvidia_ljspeech_train_clean_ngc.json
      sample_rate: 22050
      sup_data_path: /raid/LJSpeech/supplementary
      sup_data_types:
      - align_prior_matri

[NeMo I 2022-07-27 09:11:49 features:217] PADDING: 1
[NeMo I 2022-07-27 09:11:54 save_restore_connector:243] Model FastPitchModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_en_fastpitch_align/26d7e09971f1d611e24df90c7a9d9b38/tts_en_fastpitch_align.nemo.
Copying /root/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_en_fastpitch_align/26d7e09971f1d611e24df90c7a9d9b38/tts_en_fastpitch_align.nemo to ./
--2022-07-27 09:11:54--  https://raw.githubusercontent.com/nvidia/NeMo/main/examples/tts/fastpitch_finetune.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1751 (1.7K) [text/plain]
Saving to: ‘fastpitch_finetune.py’

fastpitch_finetune. 100%[===================>]   1.71K  --.-KB/s    in 0s      

2022-07-27 09:11:55 (35.0 MB/s) - ‘fastpitch_

In [8]:
from nemo.collections.tts.torch.g2ps import EnglishG2p
from nemo.collections.tts.torch.data import TTSDataset
from nemo_text_processing.text_normalization.normalize import Normalizer
from nemo.collections.tts.torch.tts_tokenizers import EnglishPhonemesTokenizer, EnglishCharsTokenizer

import torch
from tqdm.notebook import tqdm


# Text normalizer
text_normalizer = Normalizer(
    lang="en", 
    input_case="cased", 
    whitelist="./tts_dataset_files/lj_speech.tsv"
)

text_normalizer_call_kwargs = {
    "punct_pre_process": True,
    "punct_post_process": True
}

# Text tokenizer
text_tokenizer = EnglishCharsTokenizer()

def pre_calculate_supplementary_data(sup_data_path, sup_data_types, text_tokenizer, text_normalizer, text_normalizer_call_kwargs):
    # init train and val dataloaders
    stages = ["train", "val"]
    stage2dl = {}
    for stage in stages:
        ds = TTSDataset(
            manifest_filepath=f"fastpitch_{stage}.json",
            sample_rate=16000,
            sup_data_path=sup_data_path,
            sup_data_types=sup_data_types,
            n_fft=1024,
            win_length=1024,
            hop_length=256,
            window="hann",
            n_mels=80,
            lowfreq=0,
            highfreq=8000,
            text_tokenizer=text_tokenizer,
            text_normalizer=text_normalizer,
            text_normalizer_call_kwargs=text_normalizer_call_kwargs

        ) 
        stage2dl[stage] = torch.utils.data.DataLoader(ds, batch_size=1, collate_fn=ds._collate_fn, num_workers=0)

    # iteration over dataloaders
    pitch_mean, pitch_std, pitch_min, pitch_max = None, None, None, None
    for stage, dl in stage2dl.items():
        pitch_list = []
        print(stage)
        print(dl)
        for batch in tqdm(dl, total=len(dl)):
            tokens, tokens_lengths, audios, audio_lengths, attn_prior, pitches, pitches_lengths = batch
            pitch = pitches.squeeze(0)
            pitch_list.append(pitch[pitch != 0])

        if stage == "train":
            pitch_tensor = torch.cat(pitch_list)
            pitch_mean, pitch_std = pitch_tensor.mean().item(), pitch_tensor.std().item()
            pitch_min, pitch_max = pitch_tensor.min().item(), pitch_tensor.max().item()
    
    print("Pitch_mean : ",pitch_mean)
    print("Pitch_std : ",pitch_std)
    print("Pitch_min : ",pitch_min)
    print("Pitch_max : ",pitch_max)
            
    return pitch_mean, pitch_std, pitch_min, pitch_max

fastpitch_sup_data_path = "fastpitch_sup_data_folder"
sup_data_types = ["align_prior_matrix", "pitch"]

pitch_mean, pitch_std, pitch_min, pitch_max = pre_calculate_supplementary_data(
    fastpitch_sup_data_path, sup_data_types, text_tokenizer, text_normalizer, text_normalizer_call_kwargs
)

[NeMo I 2022-07-27 09:12:00 tokenize_and_classify:87] Creating ClassifyFst grammars.
[NeMo I 2022-07-27 09:12:21 data:197] Loading dataset from fastpitch_train.json.


474it [00:05, 88.36it/s]

[NeMo I 2022-07-27 09:12:27 data:232] Loaded dataset with 474 files.
[NeMo I 2022-07-27 09:12:27 data:234] Dataset contains 0.44 hours.
[NeMo I 2022-07-27 09:12:27 data:336] Pruned 0 files. Final dataset contains 474 files
[NeMo I 2022-07-27 09:12:27 data:339] Pruned 0.00 hours. Final dataset contains 0.44 hours.
[NeMo I 2022-07-27 09:12:27 data:197] Loading dataset from fastpitch_val.json.



25it [00:00, 75.42it/s]

[NeMo I 2022-07-27 09:12:27 data:232] Loaded dataset with 25 files.
[NeMo I 2022-07-27 09:12:27 data:234] Dataset contains 0.02 hours.
[NeMo I 2022-07-27 09:12:27 data:336] Pruned 0 files. Final dataset contains 25 files
[NeMo I 2022-07-27 09:12:27 data:339] Pruned 0.00 hours. Final dataset contains 0.02 hours.
train


  0%|          | 0/474 [00:00<?, ?it/s]

val


  0%|          | 0/25 [00:00<?, ?it/s]

Pitch_mean :  330.17303466796875
Pitch_std :  137.74679565429688
Pitch_min :  70.10083770751953
Pitch_max :  2057.0478515625


In [ ]:
#setting up wandb credentials
#weights and biases 
wandb_api_key = "d06dcb679ec9cf0d71c81cb750d142b6b90f3d25"
wandb_project_name = 'MLP'
wandb_run_name = character

!wandb login ${wandb_api_key}

In [ ]:
#resuming the training 
#+init_from_ptl_ckpt=Fastpitch_epoch_356.ckpt
# trainer.max_epochs=500 ~trainer.max_epochs 
#+init_from_nemo_model=tts_en_fastpitch_align.nemo \
! (python fastpitch_finetune.py --config-name=fastpitch_align_v1.05.yaml \
  train_dataset=fastpitch_train.json \
  validation_datasets=fastpitch_val.json \
  sup_data_path=fastpitch_sup_data \
  phoneme_dict_path=tts_dataset_files/cmudict-0.7b_nv22.07 \
  heteronyms_path=tts_dataset_files/heteronyms-030921 \
  whitelist_path=tts_dataset_files/lj_speech.tsv \
  exp_manager.exp_dir=training_logs \
  +init_from_nemo_model=tts_en_fastpitch_align.nemo \
  trainer.max_epochs=1000 ~trainer.max_epochs \
  trainer.check_val_every_n_epoch=7 \
  model.train_ds.dataloader_params.batch_size=8 model.validation_ds.dataloader_params.batch_size=8 \
  model.n_speakers=1 model.pitch_mean=330.173 model.pitch_std=137.747 \
  model.pitch_fmin=70.1 model.pitch_fmax=2057.05 model.optim.lr=2e-4 \
  +exp_manager.create_wandb_logger=true \
  +exp_manager.wandb_logger_kwargs.name='trixie' \
  +exp_manager.wandb_logger_kwargs.project='MLP' \
  ~model.optim.sched model.optim.name=adam trainer.devices=1 trainer.strategy=null \
  +model.text_tokenizer.add_blank_at=true \
)

## Checking the checkpoint

In [ ]:
checkpoint = ''

In [ ]:
spec_model = FastPitchModel.load_from_checkpoint(checkpointpath)
spec_model.eval().cuda()

vocoder = HifiGanModel.from_pretrained("tts_hifigan")
vocoder = vocoder.eval().cuda()

In [ ]:
custom_infer("My money don't jiggle jiggle, it folds, make your body wiggle wiggle !", spec_model2, vocoder)